In [20]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [21]:
df = pd.read_csv('../../data/process_v1/process_data_6.csv')
df = df[df['city'] == 0]

In [22]:
df['is_street_house'] = (df['accessibility'] == 2).astype(np.int32)

In [23]:
df['landSize_ratio'] = df['landSize'] / df['acreage']

In [24]:
# mean_land_size_df = df.groupby('administrative_genre')['landSize'].mean().reset_index().rename(columns = {'landSize': 'meanLandSize'})

# df = df.merge(mean_land_size_df, how='left', on = 'administrative_genre')

# df['landSize_ratio_with_administrative_genre'] = df['landSize'] / df['meanLandSize']
# df['acreage_ratio_with_meanLandSize'] = df['acreage'] / df['meanLandSize']


# del df['meanLandSize']

# def partition_for_alley_width(alley_width):
#     if math.isnan(alley_width):
#         return np.nan
#     if alley_width <= 2.5:
#         return 1
#     if alley_width <= 5:
#         return 2
#     if alley_width <= 7:
#         return 3
#     if alley_width <= 8:
#         return 4
# df['type_of_width'] = df['alley_width'].apply(partition_for_alley_width)

In [25]:
{'fitThreeCars': 0,
 'fitOneCarAndOneMotorbike': 1,
 'notInTheAlley': 2,
 'parkCar': 3,
 'theBottleNeckPoint': 4,
 'fitTwoCars': 5,
 'narrorRoad': 6}

{'fitThreeCars': 0,
 'fitOneCarAndOneMotorbike': 1,
 'notInTheAlley': 2,
 'parkCar': 3,
 'theBottleNeckPoint': 4,
 'fitTwoCars': 5,
 'narrorRoad': 6}

In [33]:
cat_cols = ['is_street_house', 'ward', 'street', 'district', 'typeOfRealEstate', 'houseDirection', 'accessibility', 'certificateOfLandUseRight'] + [c for c in df.columns.tolist() if '_lat' in c or '_lon' in c]
target_cols = ['target']
remove_cols = ['price', 'description', 'city'] + [c for c in df.columns if 'distance_hcm' in c]
num_cols = [c for c in df.columns.tolist() if c not in (cat_cols + target_cols + remove_cols)]

cat_cols = list(set(cat_cols))
target_cols = list(set(target_cols))
remove_cols = list(set(remove_cols))
num_cols = list(set(num_cols))

print(len(cat_cols), len(target_cols), len(remove_cols), len(num_cols))

28 1 52 155


In [34]:
cat_cols

['nearest_0_lon',
 'nearest_6_lat',
 'is_street_house',
 'district_lon',
 'nearest_0_lat',
 'certificateOfLandUseRight',
 'nearest_2_lon',
 'nearest_8_lat',
 'ward',
 'nearest_5_lon',
 'nearest_5_lat',
 'nearest_7_lat',
 'nearest_7_lon',
 'nearest_1_lon',
 'nearest_3_lon',
 'nearest_8_lon',
 'nearest_3_lat',
 'nearest_2_lat',
 'district_lat',
 'street',
 'accessibility',
 'nearest_4_lat',
 'nearest_1_lat',
 'houseDirection',
 'typeOfRealEstate',
 'nearest_4_lon',
 'nearest_6_lon',
 'district']

In [35]:
num_cols

['distance_hn_Công viên cá Koi Rin Rin Park',
 'num_of_fast_food_in_2000m_radius',
 'num_of_marketplace_in_1000m_radius',
 'nearest_2_district',
 'w',
 'num_of_parking_entrance_in_1000m_radius',
 'num_of_fast_food_in_500m_radius',
 'nearest_4_street',
 'num_of_ward',
 'nearest_7_ward',
 'distance_hn_Hậu Giang',
 'num_of_university_in_2000m_radius',
 'num_of_parking_in_1000m_radius',
 'num_of_parking_in_2000m_radius',
 'num_of_atm_in_2000m_radius',
 'num_of_school_in_500m_radius',
 'distance_hn_Công viên Gia Định',
 'frontWidth',
 'num_of_hospital_in_2000m_radius',
 'num_of_school_in_2000m_radius',
 'distance_nearest_4',
 'num_of_cafe_in_500m_radius',
 'h',
 'distance_hn_Công viên khu đô thị Sala',
 'num_of_university_in_500m_radius',
 'num_of_kindergarten_in_500m_radius',
 'time',
 'nearest_0_district',
 'distance_hn_Aeon Mall Shopping Center Tân Phú',
 'acreage',
 'endWidth',
 'distance_hn_Đường Nguyễn Huệ, quận 1, TPHCM',
 'distance_hn_Công viên nước Củ Chi',
 'distance_hn_Bến Vân Đồ

In [36]:
print(len(num_cols))

155


In [37]:
# train_independent_corr = df[num_cols].corr()
# train_seleted_corr_columns = np.full((train_independent_corr.shape[0],), True, dtype=bool)
# for i in range(train_independent_corr.shape[0]):
#     for j in range(i + 1, train_independent_corr.shape[0]):
#         if train_independent_corr.iloc[i, j] >= 0.99:

#             if train_seleted_corr_columns[j]:
#                 train_seleted_corr_columns[j] = False
# train_selected_columns = df[num_cols].columns[train_seleted_corr_columns]
# print(train_selected_columns.shape)

In [38]:
# train_selected_columns = list(train_selected_columns)
train_selected_columns = num_cols

In [39]:
df[cat_cols] = df[cat_cols].fillna(100)
df[cat_cols] = df[cat_cols].astype(np.int32)

In [40]:
df = df[cat_cols + num_cols + target_cols]

In [41]:
df = df[df[target_cols[0]] != np.inf]
df = df[df[target_cols[0]] != -np.inf]

In [42]:
df.shape

(49052, 184)

In [43]:
df = df[df[target_cols[0]] >= 10]
df = df[df[target_cols[0]] < 100 * 10]
# df = pd.concat([df[df['w'] < 50], df[df['w'].isnull()]])
# df = pd.concat([df[df['h'] < 50], df[df['w'].isnull()]])

In [44]:
df.to_csv('../../data/process_v1/process_data_7.csv', index = False)

In [45]:
categorical_features_indices = [i for i, c in enumerate(df.columns.tolist()) if c in cat_cols]

In [46]:
X = df.drop(target_cols[0], axis=1)
y = df[target_cols[0]] / 100

In [47]:
from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.9, random_state=42)
print(X_train.shape, y_train.shape)
print(X_validation.shape, y_validation.shape)

(39788, 183) (39788,)
(4421, 183) (4421,)


In [48]:
from catboost import CatBoostRegressor, Pool, metrics, cv
from sklearn.metrics import accuracy_score

In [50]:
from lightgbm import LGBMRegressor

In [54]:
y_train.values

array([0.51204819, 3.38723078, 2.        , ..., 1.14583333, 0.53846154,
       0.37671233])

In [63]:
model = LGBMRegressor(device = 'gpu')
model.fit(
    X_train.values[:, 5:6], y_train.values,
)

[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1


LightGBMError: GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1

In [ ]:
feature_importances = pd.DataFrame(model.get_feature_importance(),columns=['importance'])
feature = pd.DataFrame(df.columns,columns=['feature'])
feature_importance_df = pd.concat([feature_importances,feature],axis=1)
feature_importance_df[feature_importance_df['importance'] == 0]

In [ ]:
feature_importance_df[feature_importance_df['feature'] == 'alley_width']

In [ ]:
plt.figure(figsize=(16, 16))
sns.barplot(data=feature_importance_df.sort_values(by='importance', ascending=False).head(100), x='importance', y='feature')
plt.title('Feature Importances ')

In [ ]:
val_pred_df = pd.DataFrame()
val_pred_df['pred'] = model.predict(X_validation)
val_pred_df['target'] = y_validation


In [ ]:
val_pred_df

In [ ]:
def plot_distribution():
    plt.subplots(figsize=(14,7))
    sns.distplot(x=val_pred_df['target'], color='blue', kde=True)
    sns.distplot(x=val_pred_df['pred'], color='red', kde=True)
    plt.title("Distribution", weight="bold",fontsize=20, pad=20)
    plt.ylabel("Count", weight="bold", fontsize=15)
    plt.xlabel(f'Distribution', weight="bold", fontsize=12)
    plt.legend(['target', 'pred'])
    plt.show()
plot_distribution()